In [1]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
from src import utils, load_data
from src.models import lth_maml as lth
from tqdm.notebook import tqdm
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision
from torch.utils.tensorboard import SummaryWriter
from collections import namedtuple
from src import utils

print(f"found gpu: {torch.cuda.is_available()}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

found gpu: True


In [2]:
lottery_ticket_params = {
    "prune_strategy": {
        "name": "early_bird",
        "rate": .1,
        "iterations": 35,
    },
    "seeds": {
        "torch": 222,
        "cuda": 222,
        "numpy": 222
    },
    "expr_id": utils.new_expr_id("lth maml"),
    "model_training_params": {
        "training_iterations": 60000, #epoch
        "n_way": 5,                        # number of classes to choose between for each task
        "k_spt": 1,                        # k shot for support set (number of examples per class per task)
        "k_qry": 15,                       # k shot for query set (number of examples per class per task)
        "imgsz": 84,                       # image size
        "imgc": 3,                         # this isn't used anywhere????? no idea what it does???? they say its supposed to be 1 or 3...
        "task_num": 4,                     # meta model batch size
        "meta_lr": 1e-3,                   # meta model learning rate
        "update_lr": 0.01,                 # task specific model learning rate
        "update_step": 5,                  # task specific model training epochs
        "update_step_test": 10,            # task specific model testing epochs
#         "optimizer": ("adam", {"lr": 0.0001}),
#         "loss_func": "cross_entropy",
        "model_name": "MAML",
        "dataset_name": "mini_imagenet",
        "layer_definitions": None
    }
}

lottery_ticket_params["model_training_params"]["layer_definitions"] = [
    ('conv2d', [32, 3, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 2, 0]),
    ('conv2d', [32, 32, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 2, 0]),
    ('conv2d', [32, 32, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 2, 0]),
    ('conv2d', [32, 32, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 1, 0]),
    ('flatten', []),
    ('linear', [lottery_ticket_params["model_training_params"]["n_way"], 32 * 5 * 5]) # 32 * 5 * 5
]
utils.set_seeds(lottery_ticket_params["seeds"])

In [3]:
args = lottery_ticket_params["model_training_params"]
dataset = load_data.mini_imagenet(args)

# train, val, test = dataset

# images_support, labels_support, images_query, labels_query = next(iter(train))
# print(f"num train batches: {len(train)}")
# print(f"images_support {images_support.shape}")
# print(f"labels_support {labels_support.shape}")
# print(f"images_query {images_query.shape}")
# print(f"labels_query {labels_query.shape}")

# images = images_query.flatten(0, 1)
# labels = labels_query.flatten(0, 1)
# for i in range(len({i.numpy().item() for i in labels})):
#     grid = torchvision.utils.make_grid(images[labels == i])
#     with SummaryWriter(f'tensorboard/lth_maml_miniimagenet') as writer:
#         writer.add_image('processed', grid, i)
#         writer.flush()

# images_support, labels_support, images_query, labels_query = next(iter(val))
# print(f"num val batches: {len(val)}")
# print(f"images_support {images_support.shape}")
# print(f"labels_support {labels_support.shape}")
# print(f"images_query {images_query.shape}")
# print(f"labels_query {labels_query.shape}")

# images_support, labels_support, images_query, labels_query = next(iter(test))
# print(f"num test batches: {len(test)}")
# print(f"images_support {images_support.shape}")
# print(f"labels_support {labels_support.shape}")
# print(f"images_query {images_query.shape}")
# print(f"labels_query {labels_query.shape}")

shuffle DB :train, b:10000, 5-way, 1-shot, 15-query, resize:84
shuffle DB :val, b:100, 5-way, 1-shot, 15-query, resize:84
shuffle DB :test, b:100, 5-way, 1-shot, 15-query, resize:84


In [5]:
prune_strategy = lottery_ticket_params["prune_strategy"]
training_params = lottery_ticket_params["model_training_params"]
prune_rate = prune_strategy["rate"]
train_data, val_data, test_data = dataset

# building model
images, labels = next(iter(train_data))
input_shape = images.shape[1:]
model = build_model(training_params, input_shape)

# saving initial model weights
initial_weights = {n: w.cpu().detach() for n, w in model.state_dict().items()}
mask = build_mask(model, prune_strategy)

ValueError: too many values to unpack (expected 2)

In [ ]:
# print(f"starting lth run {expr_params['expr_id']}")
# mask = lth.run(dataset, expr_params)

starting lth run expr 2020-11-05 | 16:18:53 lth maml I4I3T
starting prune iteration 0


train epoch 0


step: 0 	training acc: [0.21666667 0.25333333 0.26333333 0.26333333 0.26333333 0.26333333]
validating model...
val acc: [0.2039 0.2418 0.2422 0.2423 0.2432 0.2427 0.2428 0.243  0.2439 0.2441
 0.2451]
step: 30 	training acc: [0.14333333 0.24333333 0.24666667 0.25666667 0.26       0.26666667]
step: 60 	training acc: [0.20333333 0.28333333 0.29333333 0.29666667 0.29333333 0.29666667]
step: 90 	training acc: [0.20666667 0.32666667 0.34       0.34666667 0.34666667 0.34666667]
step: 120 	training acc: [0.23333333 0.33       0.34       0.36       0.36       0.36666667]
step: 150 	training acc: [0.23       0.28666667 0.29       0.28666667 0.28333333 0.28      ]
step: 180 	training acc: [0.21       0.35333333 0.36       0.37       0.37666667 0.38      ]
step: 210 	training acc: [0.22       0.37333333 0.38333333 0.37666667 0.37666667 0.37      ]
step: 240 	training acc: [0.19       0.31       0.34666667 0.34666667 0.33333333 0.33      ]
step: 270 	training acc: [0.21333333 0.34333333 0.37333333 